In [1]:
%reload_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [3]:
# wdir = "chains/"
# file = "LCDM_phy_SN_nested_dynesty_multi_1.txt"
wdir = "data/"
file = "AT.dat"


data = np.loadtxt(wdir+file)
# # samples
# X = data[:,2:5]
# # likes
# Y = data[:,1]
X = data[:, :3]
Y = data[:, 3]
lenx, input_shape = np.shape(X)

In [4]:
psplit = 0.8
ntrain = int(psplit* len(X))
indx = [ntrain]
shuffle = np.random.permutation(lenx)
X = X[shuffle]
Y = Y[shuffle]
X_train, X_test = np.split(X, indx)
Y_train, Y_test = np.split(Y, indx)

In [5]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0,
                                   patience=10,
                                   restore_best_weights=True)]

In [6]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([30, 50, 100, 150]))
# HP_NUM_UNITS5 = hp.HParam('num_units5', hp.Discrete([50, 100, 200]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.0))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([4, 8]))

In [7]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_BATCHSIZE, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric('loss', display_name="Loss")])
    


In [8]:
epochs = 100
# input_z = Input(shape = (1,))


In [9]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
#         tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], activation='relu'),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS3], activation='relu'),
#         tf.keras.layers.Dense(hparams[HP_NUM_UNITS4], activation='linear'),
#         tf.keras.layers.Dense(hparams[HP_NUM_UNITS5], activation='linear'),
        tf.keras.layers.Dense(1, activation='linear')
    ])
                              
    model.compile(
            optimizer=hparams[HP_OPTIMIZER],
#             optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(X_train, Y_train, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], verbose=0) 
    _, loss = model.evaluate(X_test, Y_test)


    return loss
    

In [10]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [11]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
#                 for num_units4 in HP_NUM_UNITS4.domain.values:
#                     for num_units5 in HP_NUM_UNITS5.domain.values:
                for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
                    for optimizer in HP_OPTIMIZER.domain.values:
                        hparams = {
                            HP_NUM_UNITS1: num_units1,
                            HP_NUM_UNITS2: num_units2,
                            HP_NUM_UNITS3: num_units3,
                            #                       HP_NUM_UNITS4: num_units4,
                            #                           HP_NUM_UNITS5: num_units5,
                            HP_DROPOUT: dropout_rate,
                            HP_OPTIMIZER: optimizer,
                            HP_BATCHSIZE: batch_size
                        }
                        run_name = "run-%d" % session_num
                        print('--- Starting trial: %s' % run_name)
                        print({h.name: hparams[h] for h in hparams})
                        run('logs/hparam_tuning/' + run_name, hparams)
                        session_num += 1

print(session_num)

--- Starting trial: run-0
{'num_units1': 50, 'num_units2': 50, 'num_units3': 50, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 4}
54/54 [==============================] - 0s 431us/step - loss: 2.4130e-04 - mean_squared_error: 2.4130e-04
--- Starting trial: run-1
{'num_units1': 50, 'num_units2': 50, 'num_units3': 50, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 4}
54/54 [==============================] - 0s 439us/step - loss: 1.3056e-04 - mean_squared_error: 1.3056e-04
--- Starting trial: run-2
{'num_units1': 50, 'num_units2': 50, 'num_units3': 100, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 4}
54/54 [==============================] - 0s 440us/step - loss: 2.0586e-04 - mean_squared_error: 2.0586e-04
--- Starting trial: run-3
{'num_units1': 50, 'num_units2': 50, 'num_units3': 100, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 4}
54/54 [==============================] - 0s 473us/step - loss: 2.8651e-04 - mean_squared_error: 2.8651e-04
--- Starting trial: run-4
{'nu

54/54 [==============================] - 0s 413us/step - loss: 3.7125e-04 - mean_squared_error: 3.7125e-04
--- Starting trial: run-34
{'num_units1': 100, 'num_units2': 150, 'num_units3': 150, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 4}
54/54 [==============================] - 0s 427us/step - loss: 1.4787e-04 - mean_squared_error: 1.4787e-04
--- Starting trial: run-35
{'num_units1': 100, 'num_units2': 150, 'num_units3': 150, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 4}
54/54 [==============================] - 0s 462us/step - loss: 1.6441e-04 - mean_squared_error: 1.6441e-04
--- Starting trial: run-36
{'num_units1': 150, 'num_units2': 50, 'num_units3': 50, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 4}
54/54 [==============================] - 0s 463us/step - loss: 1.3474e-04 - mean_squared_error: 1.3474e-04
--- Starting trial: run-37
{'num_units1': 150, 'num_units2': 50, 'num_units3': 50, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 4}
54/54 [=============

54/54 [==============================] - 0s 384us/step - loss: 9.3455e-05 - mean_squared_error: 9.3455e-05
--- Starting trial: run-68
{'num_units1': 50, 'num_units2': 150, 'num_units3': 100, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 8}
54/54 [==============================] - 0s 399us/step - loss: 7.8249e-05 - mean_squared_error: 7.8249e-05
--- Starting trial: run-69
{'num_units1': 50, 'num_units2': 150, 'num_units3': 100, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 8}
54/54 [==============================] - 0s 416us/step - loss: 6.1826e-05 - mean_squared_error: 6.1826e-05
--- Starting trial: run-70
{'num_units1': 50, 'num_units2': 150, 'num_units3': 150, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 8}
54/54 [==============================] - 0s 400us/step - loss: 1.5604e-04 - mean_squared_error: 1.5604e-04
--- Starting trial: run-71
{'num_units1': 50, 'num_units2': 150, 'num_units3': 150, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 8}
54/54 [=============

54/54 [==============================] - 0s 392us/step - loss: 1.3833e-04 - mean_squared_error: 1.3833e-04
--- Starting trial: run-101
{'num_units1': 150, 'num_units2': 100, 'num_units3': 150, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 8}
54/54 [==============================] - 0s 446us/step - loss: 2.4271e-04 - mean_squared_error: 2.4271e-04
--- Starting trial: run-102
{'num_units1': 150, 'num_units2': 150, 'num_units3': 50, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 8}
54/54 [==============================] - 0s 432us/step - loss: 2.7011e-05 - mean_squared_error: 2.7011e-05
--- Starting trial: run-103
{'num_units1': 150, 'num_units2': 150, 'num_units3': 50, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 8}
54/54 [==============================] - 0s 391us/step - loss: 1.2485e-04 - mean_squared_error: 1.2485e-04
--- Starting trial: run-104
{'num_units1': 150, 'num_units2': 150, 'num_units3': 100, 'dropout': 0.0, 'optimizer': 'adam', 'batch_size': 8}
54/54 [=======

In [12]:
rm -rf /tmp/tb_logs/

In [13]:
# !kill 16011

In [14]:
%tensorboard --logdir logs/hparam_tuning